In [129]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from scipy.stats import norm

In [130]:
def Asian_Analytical(T, K, r, S0, sigma, N, trials):
    
    '''
    Calculate Asian Option 
    Analytical expression for Asian call option based on geometric averages
    '''
    sigma_tilde = sigma * np.sqrt((2*N + 1)/(6*(N+1)))
    r_tilde = ((r - 0.5*sigma*sigma) + (sigma_tilde*sigma_tilde))/2
    d1 = (np.log(S0/K) + (r_tilde + 0.5*sigma_tilde**2)*T) /(np.sqrt(T)*sigma_tilde)
    d2 = (np.log(S0/K) + (r_tilde - 0.5*sigma_tilde**2)*T) /(np.sqrt(T)*sigma_tilde)
    
    
    return np.exp(-r*T) * (S0 * np.exp(r_tilde *T)*norm.cdf(d1) - K*norm.cdf(d2))

In [131]:
Asian_Analytical(1, 29, 0.08, 30, 0.3, 250,0)

2.9658572077909873

In [70]:
@jit(nopython=False, fastmath = True)
def GBM_Euler(T, K, S, sigma, r, M):
    '''
    Inputs: Time, Strike price, asset price, vol, interest rate, number of steps
    '''
    
    dt = T/M
    S_all = np.zeros(M)
    S_all[0] = S
    
    for i in range(1, M):
        S_all[i] = S_all[i-1] + r* S_all[i-1] * dt + sigma * S_all[i-1] * np.sqrt(dt) * np.random.normal()
    return S_all

In [309]:
T = 1; K = 29; r = 0.08; S = 30; sigma = 0.3; trials = 1000; M = 100

In [312]:
# https://stackoverflow.com/questions/43099542/python-easy-way-to-do-geometric-mean-in-python
def geo_mean_overflow(iterable):
    return  np.exp(np.log(iterable).mean())

In [320]:

'''
Monte Carlo for Geometric Average of Asian 
'''
s = np.zeros(trials)

for i in range(trials ):
    test = GBM_Euler(T, K, S, sigma, r, M)
#    s[i] = np.max((0, (test.prod())**(1/(len(test))) - K ))
    s[i] = np.max((0, geo_mean_overflow(test)   - K))
    
print(f"Final: {np.mean(s)}")
CI = [np.mean(s) - 1.96*np.std(s)/np.sqrt(len(s)), np.mean(s) + 1.96*np.std(s)/np.sqrt(len(s))]
print(f"Confidence interval: {CI[0], CI[1]}")
print(f"Size of CI: {CI[1]-CI[0]}")
                                                        

Final: 3.2090233716243226
Confidence interval: (2.9590144285790365, 3.4590323146696087)
Size of CI: 0.5000178860905722


In [ ]:
"""
Monte Carlo for Arithmetic Average of Asian Call option
"""
s = np.zeros(trials)

for i in range(trials):
    test = GBM_Euler(T, K, S, sigma, r, M)
    S_arith = (1/(len(test)) * np.sum(test)
    s[i] = np.max(0, S_arith - K)
    CI = [np.mean(s) - 1.96*np.std(s)/np.sqrt(len(s)), np.mean(s) + 1.96*np.std(s)/np.sqrt(len(s))]
    

In [322]:
test

array([30.        , 31.59936853, 31.75889882, 31.19830527, 29.80312879,
       30.51868252, 31.95203535, 31.1481042 , 31.37664757, 31.7733514 ,
       30.86962378, 31.07338869, 30.99347717, 31.80494323, 31.09840319,
       31.90733173, 31.75095601, 30.47047141, 30.0225035 , 30.03974034,
       29.3644511 , 29.28394265, 31.27595144, 31.37713105, 31.15621967,
       30.01606402, 29.44471601, 29.72459477, 30.44249432, 30.64175155,
       31.34779996, 30.83151793, 31.72267108, 31.97699709, 31.62093842,
       31.15570863, 31.93208916, 32.56866148, 31.27710963, 30.41775062,
       29.45314912, 29.36835467, 29.3741725 , 29.89565884, 30.70120163,
       30.35467881, 30.55259877, 32.16380606, 32.59752364, 32.58970758,
       34.15164502, 34.30189298, 33.63518947, 34.08164822, 32.13445599,
       29.45460657, 30.4756572 , 30.14693618, 30.01484922, 31.00233019,
       30.99082575, 28.80044223, 27.40827903, 25.83373751, 25.89880062,
       25.31457851, 26.87197054, 27.73184587, 27.91059344, 28.37

In [323]:
s

array([2.11811868e+00, 0.00000000e+00, 1.96649755e+01, 4.68313244e+00,
       6.18964925e+00, 2.59342764e+00, 0.00000000e+00, 9.03119927e+00,
       9.78205581e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.43621913e+00, 9.26951408e+00, 0.00000000e+00,
       6.20811272e+00, 9.05418951e+00, 0.00000000e+00, 1.81685636e+00,
       2.88390493e+00, 6.97839671e+00, 2.13575904e+00, 2.01559830e+00,
       0.00000000e+00, 8.34575481e+00, 2.78017948e+00, 7.44083628e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.19690115e+00,
       9.82183456e+00, 0.00000000e+00, 1.41617569e+00, 8.35081299e+00,
       0.00000000e+00, 2.61345613e+00, 0.00000000e+00, 0.00000000e+00,
       6.22611010e-01, 0.00000000e+00, 0.00000000e+00, 1.58208363e+00,
       8.59641249e+00, 3.99056649e-01, 4.59572512e-01, 8.83683622e+00,
       0.00000000e+00, 2.34923021e+00, 6.42387929e-01, 5.29683587e-01,
       9.53612921e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [ ]:
np.max((0, ))